# Projectile Motion Lab

In elementary physics courses, one often sees the projectile motion problem without air resistance, for which the solution is fully analytic and straightforward:
* The maximum range occurs at a $45^\circ$ launch angle,
* The horizontal range is given by the well-known formula $R = (v_0^2/g) \sin(2\alpha)$.

However, if we include **air resistance**, the situation becomes more complicated.

This lab shows:
1. No Air Resistance: Simple closed-form solution and a quick check via **gradient descent**.
2. Linear Air Resistance: An analytic solution for $x(t)$ and $y(t)$ is still possible, although the time-of-flight can only be solved implicitly (involving a transcendental equation).
   We can still implement a **gradient descent** on the angle $\alpha$, using these closed-form expressions plus a (small) numerical step to find the time of flight.